###################################################################       
#Script Name    :                                                                                              
#Description    :                                                                                 
#Args           :                                                                                           
#Author         : Nikhil Rao in R, converted to Python by Nor Raymond                                              
#Email          : nraymond@appen.com                                          
###################################################################

In [8]:
import os
import glob 
import pandas as pd
import numpy as np
import yaml
from IPython.core.display import display, HTML

In [9]:
# Function to load yaml configuration file
def load_config(config_name):
    with open(os.path.join(config_path, config_name), 'r') as file:
        config = yaml.safe_load(file)

    return config

config_path = "conf/base"

try:
    
    # load yaml catalog configuration file
    config = load_config("catalog.yml")

    os.chdir(config["project_path"])
    root_path = os.getcwd()
    
except:
    
    os.chdir('..')
    # load yaml catalog configuration file
    config = load_config("catalog.yml")

    os.chdir(config["project_path"])
    root_path = os.getcwd()

In [10]:
# import data_cleaning module
import src.data.data_cleaning as data_cleaning

### Functions to initialize data ingestion

In [11]:
def group_files_by_language(data_path, files, file_initials):
    
    file_groups = {}  
    for x in files:  
        key = x.split('_')[0] #x[:16] # The key is the first 16 characters of the file name
        group = file_groups.get(key,[])
        group.append(x)  
        file_groups[key] = group
                
    return file_groups

def create_file_exists_df(files, file_initials):
    
    checker = []
    file_exists = []
    for fname in files:
        for key in file_initials:
            if key in fname:
                file_exists.append((key, fname))

    file_exists = pd.DataFrame(file_exists, columns =['Keyword', 'Filename'])
    
    return file_exists

def data_ingestion_initialize(root_path, run_value, run_value_2):
    
    # Function to load yaml configuration file
    def load_config(config_name):
        with open(os.path.join(config_path, config_name), 'r') as file:
            config = yaml.safe_load(file)

        return config

    # load yaml catalog configuration file
    config = load_config("catalog.yml")
    
    # define reference file paths
    ref_path = os.path.join(root_path, config["data_path"]["ref"])
    ref_filepath = os.path.join(ref_path, config["filenames"]["rc_col_ref"])
    ref_data = pd.read_excel(io = ref_filepath, sheet_name="threshold_raters", header=None)
    
    if len(ref_data) != 0:
        ref_data_cols = ref_data[0].tolist()
    else:
        ref_data_cols = []

    print("Initialize data ingestion and file checking...\n")
    
    if run_value == 'Deployment':
        
        # define data input paths
        data_path = os.path.join(root_path, config["data_path"]["output"], 'Deployment')
        survey_path = ''
        
    else:
        
        # define data input paths
        data_path = os.path.join(root_path, config["data_path"]["output"], run_value, run_value_2)
        survey_path = os.path.join(root_path, config["data_path"]["survey"])
       
    # get the list of files in raw folder
    files = os.listdir(data_path)
    files = [f for f in files if f[-5:] == '.xlsx']
    
    file_initials = ['RC', 'Vocab_1', 'Vocab_2']

    languages = []
    for file in files:
        for file_initial in file_initials:        
            lang = file.split('_' + file_initial)[0]
        if not lang.endswith((".xlsx")):
            languages.append(lang)
    
    languages = pd.DataFrame(languages, columns = ['Language'])
    
    file_groups = group_files_by_language(data_path, files, file_initials)
    
    file_exists = create_file_exists_df(files, file_initials)
    
    return data_path, files, languages, file_groups, file_exists, ref_data_cols, survey_path
       

### Functions for data processing - DEPLOY

In [12]:
file_initials = ['RC', 'Vocab_1', 'Vocab_2']

def obtain_file_summary_df(file_initials, file_exists, data_path):
    
    df_summary = []
    for k in file_initials:
        selected_files = file_exists[file_exists['Keyword'] == k] 
        selected_filenames = selected_files['Filename'].tolist()

        df = pd.DataFrame()
        for f in selected_filenames:
            data = pd.read_excel(os.path.join(data_path, f), 'Summary')
            df = df.append(data)

        df_summary.append(df)    
        
    return df_summary

def obtain_file_data_df(file_initials, file_exists, data_path):
    
    df_data = []
    for k in file_initials:
        selected_files = file_exists[file_exists['Keyword'] == k] 
        selected_filenames = selected_files['Filename'].tolist()

        df = pd.DataFrame()
        for f in selected_filenames:
            data = pd.read_excel(os.path.join(data_path, f), 'Data')
            df = df.append(data)

        df_data.append(df)    
        
    return df_data

def obtain_distinct_raters(df_summary, ref_data_cols):

    r1 = df_summary[0] # Joined data for Summary sheet from RC 
    r2 = df_summary[1] # Joined data for Summary page from Vocab_1 
    r3 = df_summary[2] # Joined data for Summary page from Vocab_2 
             
    raters = pd.concat([r1,r2,r3], ignore_index=True)
    raters = raters[['_worker_id', 'Grouping', 'Market', 'Language']]
    raters = raters.drop_duplicates()
    
    if len(ref_data_cols) != 0:
        
        threshold_raters = ref_data_cols
        raters = raters[raters['_worker_id'].isin(threshold_raters)]
    
    # obtain languages from r1 and create a dataframe
    languages = r1.Language.unique().tolist()
    languages = pd.DataFrame(languages, columns = ['Language'])
    
    return raters, r1, r2, r3, languages

def merge_raters_to_df_data(df_data, raters):

    rc = df_data[0] # Joined data for Data sheet from RC 
    v1 = df_data[1] # Joined data for Data page from Vocab_1 
    v2 = df_data[2] # Joined data for Data page from Vocab_2 
    
    # Merge raters to v1, v2, and rc
    rc = pd.merge(rc, raters,  how='left', on=['_worker_id', 'Language'])
    v1 = pd.merge(v1, raters,  how='left', on=['_worker_id', 'Language'])
    v2 = pd.merge(v2, raters,  how='left', on=['_worker_id', 'Language'])
    
    # Convert _created_at and _started_at to date-time
    rc[['_created_at','_started_at']] = rc[['_created_at','_started_at']].apply(pd.to_datetime, format='%m/%d/%Y %H:%M:%S')
    v1[['_created_at','_started_at']] = v1[['_created_at','_started_at']].apply(pd.to_datetime, format='%m/%d/%Y %H:%M:%S')
    v2[['_created_at','_started_at']] = v2[['_created_at','_started_at']].apply(pd.to_datetime, format='%m/%d/%Y %H:%M:%S')

    return rc, v1, v2


### Functions for data processing - PILOT

In [18]:
def survey_selection(root_path, config):

    survey_path = os.path.join(root_path, config["data_path"]["survey"])
       
    # get the list of files in raw folder
    files = os.listdir(survey_path)
    files = [f for f in files if f[-5:] == '.xlsx']
    
    survey_files = pd.DataFrame(files, columns = ['Survey Filename'])
    
    print(survey_files)
    
    while True:
        try:
            
            survey_index = int(input("\nPlease select the number of the survey filename for your pilot run: "))
            if survey_index < min(survey_files.index) or survey_index > max(survey_files.index):
                print(f"\nYou must enter numbers between {min(survey_files.index)} - {max(survey_files.index)}... Please try again")
                continue
            elif survey_index == "":
                print("\nYou must enter any numbers")
                continue
            else:
                print(f"\nYou have selected {survey_index} for '{survey_files.iloc[survey_index, 0]}'\n")
                survey_selected = survey_files.iloc[survey_index, 0]
                break

        except ValueError:
            print(f"\nYou must enter numerical values only... Please try again")
            continue
        else:
            break
    
    return survey_selected

In [16]:
def obtain_survey_fluency(survey_data):
    
    fluency = []
    for opt in survey_data['31_language_1']:
        if opt == 'over_15_years' :
            fluency.append('Fluent')
        elif opt == '1015_years' :
            fluency.append('Fluent')
        elif opt == '510_years' :
            fluency.append('Intermediate')
        elif opt == '03_years' :
            fluency.append('Not Fluent')   
        else:
            fluency.append('') 
            
    return fluency

def obtain_survey_data(survey_path, survey_selected):
         
    survey_data = pd.read_excel(os.path.join(survey_path, survey_selected), 'Sheet1')
    try:
        survey_data = survey_data.drop('Unnamed: 42', axis = 1)
        survey_data[['_created_at','_started_at']] = survey_data[['_created_at','_started_at']].apply(pd.to_datetime, format='%m/%d/%Y %H:%M:%S')
        survey_data = survey_data.rename(columns = {"_created_at" : "survey_created_at", "_started_at" : "survey_started_at"})
        survey_data = survey_data[['_worker_id', '31_language_1', 'survey_created_at', 'survey_started_at']]
        survey_data['Fluency'] = obtain_survey_fluency(survey_data)
    except:
        survey_data = survey_data
        survey_data[['_created_at','_started_at']] = survey_data[['_created_at','_started_at']].apply(pd.to_datetime, format='%m/%d/%Y %H:%M:%S')
        survey_data = survey_data.rename(columns = {"_created_at" : "survey_created_at", "_started_at" : "survey_started_at"})
        survey_data = survey_data[['_worker_id', '31_language_1', 'survey_created_at', 'survey_started_at']]
        survey_data['Fluency'] = obtain_survey_fluency(survey_data)
            
    return survey_data

def merge_to_survey_data(df_data, raters, survey_data):
    
    rc = df_data[0] # Joined data for Data sheet from RC 
    v1 = df_data[1] # Joined data for Data page from Vocab_1 
    v2 = df_data[2] # Joined data for Data page from Vocab_2 
    
    # Merge raters data to v1, v2, and rc
    rc = pd.merge(rc, raters,  how='left', left_on=['_worker_id'], right_on=['_worker_id'])
    v1 = pd.merge(v1, raters,  how='left', left_on=['_worker_id'], right_on=['_worker_id'])
    v2 = pd.merge(v2, raters,  how='left', left_on=['_worker_id'], right_on=['_worker_id'])
    
    # Merge raters data to v1, v2, and rc
    rc = pd.merge(rc, survey_data,  how='left', left_on=['_worker_id'], right_on=['_worker_id'])
    v1 = pd.merge(v1, survey_data,  how='left', left_on=['_worker_id'], right_on=['_worker_id'])
    v2 = pd.merge(v2, survey_data,  how='left', left_on=['_worker_id'], right_on=['_worker_id'])
    
    # Drop duplicat cols
    rc = rc.drop(['Language_y', 'Market'], axis = 1)
    v1 = v1.drop(['Language_y', 'Market'], axis = 1)
    v2 = v2.drop(['Language_y', 'Market'], axis = 1)
    
    rc = rc.rename(columns = {"Language_x":"Language"})
    v1 = v1.rename(columns = {"Language_x":"Language"})
    v2 = v2.rename(columns = {"Language_x":"Language"})
    
    rc['Fluency'] = np.where(rc['Grouping'] == 'GT', 'GT', rc['Fluency'])
    v1['Fluency'] = np.where(v1['Grouping'] == 'GT', 'GT', v1['Fluency'])
    v2['Fluency'] = np.where(v2['Grouping'] == 'GT', 'GT', v2['Fluency'])
    
    rc['Fluency'] = np.where(rc['Fluency'].isna(), 'Fluent', rc['Fluency'])
    v1['Fluency'] = np.where(v1['Fluency'].isna(), 'Fluent', v1['Fluency'])
    v2['Fluency'] = np.where(v2['Fluency'].isna(), 'Fluent', v2['Fluency'])
    
    # Convert _created_at and _started_at to date-time
    rc[['_created_at','_started_at']] = rc[['_created_at','_started_at']].apply(pd.to_datetime, format='%m/%d/%Y %H:%M:%S')
    v1[['_created_at','_started_at']] = v1[['_created_at','_started_at']].apply(pd.to_datetime, format='%m/%d/%Y %H:%M:%S')
    v2[['_created_at','_started_at']] = v2[['_created_at','_started_at']].apply(pd.to_datetime, format='%m/%d/%Y %H:%M:%S')
    
    return rc, v1, v2


In [19]:
def main():

    file_initials = ['RC', 'Vocab_1', 'Vocab_2']
    
    language, market, run_value, run_value_2  = data_cleaning.main()
    
    data_path, files, languages, file_groups, file_exists, ref_data_cols, survey_path = data_ingestion_initialize(root_path, run_value, run_value_2)
    
    df_summary = obtain_file_summary_df(file_initials, file_exists, data_path)
    df_data = obtain_file_data_df(file_initials, file_exists, data_path)
    raters, r1, r2, r3, languages =  obtain_distinct_raters(df_summary, ref_data_cols)
        
    if run_value == 'Deployment':
        
        rc, v1, v2 = merge_raters_to_df_data(df_data, raters)
        
    else: 
        
        survey_selected = survey_selection(root_path, config)
        survey_data = obtain_survey_data(survey_path, survey_selected)
        rc, v1, v2 = merge_to_survey_data(df_data, raters, survey_data)
    
    return raters, r1, r2, r3, languages, rc, v1, v2, run_value, run_value_2 


if __name__ == "__main__":

    raters, r1, r2, r3, languages, rc, v1, v2, run_value, run_value_2  = main()
    print(languages)
    
    print('\nAutomated data processing completed.')

Initialize data ingestion and file checking...

PASS: All files exists!



Please input the type of run e.g. Deployment, Pilot 1, Pilot 2, Pilot 3 .... etc.:  Pilot 3



Run type: Pilot 3



Please input the pilot subfolder name e.g. Pilot 1A, Pilot 2C, Pilot 3A-B .... etc.:  Pilot 3A



Pilot subfolder: Pilot 3A



Do you know the 'Language' and/or 'Market code' for this file? (y/n) :  y

Please enter the Language:  Chinese-Traditional

Please enter the Market code: eg. EN-EN for English :  ZH-ZH



Starting automated data cleaning....

Dataframe created from RC file
Language and Market columns and values inserted to 'Summary' sheet
Language column and values inserted to 'Data' sheet
Missing columns inserted into 'Data' sheet.

Preview cleaned datasets:



df_summary_cleaned


              Language Market  _worker_id  Score  Percentage  Grouping
0  Chinese-Traditional  ZH-ZH    45488857     19    0.791667  Pilot 3A
1  Chinese-Traditional  ZH-ZH    45492033     21    0.875000  Pilot 3A
2  Chinese-Traditional  ZH-ZH    45492267     21    0.875000  Pilot 3A
3  Chinese-Traditional  ZH-ZH    45492778     20    0.833333  Pilot 3A
4  Chinese-Traditional  ZH-ZH    45493055     19    0.791667  Pilot 3A


df_data_cleaned


              Language           _id question_no_1 question_no_2  \
0  Chinese-Traditional  5.868399e+09             a             c   
1  Chinese-Traditional  5.868408e+09             a             c   
2  Chinese-Traditional  5.868455e+09             c             c  


Please select the number of the survey filename for your pilot run:  2



You have selected 2 for 'Survey Pilot 2 and 3.xlsx'

              Language
0  Chinese-Traditional

Automated data processing completed.
